In [2]:
from glob import glob
from pismragis.processing import convert_netcdf_to_dataframe, ncfile2dataframe

In [ ]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = convert_netcdf_to_dataframe(infiles, add_vars=False)
df.to_parquet("../tests/data/test_scalar.parquet")
df.to_csv("../tests/data/test_scalar.csv")



In [ ]:
df = convert_netcdf_to_dataframe(infiles, add_vars=False, resample="yearly")
df.to_parquet("../tests/data/test_scalar_YM.parquet")
df.to_csv("../tests/data/test_scalar_YM.csv")



In [ ]:
df = convert_netcdf_to_dataframe(infiles, add_vars=True)
df.to_parquet("../tests/data/test_scalar_add_vars.parquet")



In [1]:
infile = "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc"

In [8]:
df = ncfile2dataframe(infile, add_vars=False)
df.to_parquet("../tests/data/test_scalar_file.parquet")
df.to_csv("../tests/data/test_scalar_file.csv")

In [10]:
df = ncfile2dataframe(infile, add_vars=False, resample="yearly")
df.to_parquet("../tests/data/test_scalar_file_YM.parquet")
df.to_csv("../tests/data/test_scalar_file_YM.csv")

In [11]:
df = ncfile2dataframe(infile, add_vars=True)
df.to_parquet("../tests/data/test_scalar_file_add_vars.parquet")
df.to_csv("../tests/data/test_scalar_file_add_vars.csv")